In [242]:
from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split as ts
import sys
#from numpy import *
#import numpy
import pandas as pd
from os import listdir
import numpy
import numpy as np
from sklearn.model_selection import train_test_split
import random

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np

from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

import time
import datetime
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [243]:
%matplotlib inline
%pylab inline
import warnings
import itertools    
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense, Flatten, LSTM, Input, Dropout, BatchNormalization, GRU
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

sns.set_style('dark')

warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [244]:
dir_path = "/content/drive/MyDrive/Emotion-RST/vrchat_emotion_data2022/MATRUE_DATA"
dirList = listdir(dir_path)

data=[]
for i in range(len(dirList)):
    data_file_path = dir_path+"/"+dirList[i]+"/"+"processed_gaze_rv_concated.csv"
    print(data_file_path)
    #data_df = pd.read_csv(data_file_path)
    data_df = pd.read_csv(data_file_path, skiprows=lambda x: x > 0 and x % 1 != 0)#每5行取一个样本
    if(dirList[i]=="2022-7-12-Wangshihao"):
      #print("Replace",type(data_df['Valence'].iloc[-1]))
      data_df['Valence'] = data_df['Valence'].replace(3,4)
      #print(data_df)
    data.append(data_df)

dataset = pd.concat(data)

/content/drive/MyDrive/Emotion-RST/vrchat_emotion_data2022/MATRUE_DATA/2022-10-18-Jianzhang/processed_gaze_rv_concated.csv
/content/drive/MyDrive/Emotion-RST/vrchat_emotion_data2022/MATRUE_DATA/2022-7-19-Wangshihao/processed_gaze_rv_concated.csv
/content/drive/MyDrive/Emotion-RST/vrchat_emotion_data2022/MATRUE_DATA/2022-10-12-Yushan/processed_gaze_rv_concated.csv
/content/drive/MyDrive/Emotion-RST/vrchat_emotion_data2022/MATRUE_DATA/2022-7-12-Wangshihao/processed_gaze_rv_concated.csv
/content/drive/MyDrive/Emotion-RST/vrchat_emotion_data2022/MATRUE_DATA/2022-7-19-Wangboman/processed_gaze_rv_concated.csv
/content/drive/MyDrive/Emotion-RST/vrchat_emotion_data2022/MATRUE_DATA/2022-10-18-Zhou/processed_gaze_rv_concated.csv
/content/drive/MyDrive/Emotion-RST/vrchat_emotion_data2022/MATRUE_DATA/2022-7-08-Yang/processed_gaze_rv_concated.csv
/content/drive/MyDrive/Emotion-RST/vrchat_emotion_data2022/MATRUE_DATA/2022-7-15-liruiheng/processed_gaze_rv_concated.csv
/content/drive/MyDrive/Emotion-R

In [245]:

from sklearn.utils import shuffle
df = shuffle(dataset)
df = df.fillna(0)
df

,Position_x_mean,Position_x_std,Position_x_min,Position_x_25%,Position_x_50%,Position_x_75%,Position_x_max,Position_y_mean,Position_y_std,Position_y_min,...,Valence,Arousal,Dominance,Excite-Not Excite,Serious-Not Serious,Smooth-Not Smooth,Bright-Not Bright,Eminent-Not Eminent,Close-Not Close,Equal-Not Equal
1033,0.171388,0.010969,0.151301,0.159677,0.174990,0.180601,0.186221,1.633835,0.025648,1.599373,...,5,9,3,1,4,1,4,5,1,1
360,0.210466,0.006612,0.187683,0.208756,0.211959,0.215869,0.217101,1.118639,0.003394,1.114959,...,3,5,2,3,3,3,3,3,3,3
297,0.799668,0.002757,0.791855,0.798442,0.799167,0.800182,0.812990,1.659299,0.009905,1.634763,...,5,5,5,4,3,4,3,3,4,4
1038,0.193868,0.010371,0.171585,0.187967,0.188632,0.206787,0.209967,1.430299,0.001114,1.426841,...,7,6,5,3,3,3,3,3,3,3
762,0.250851,0.041102,0.167235,0.265717,0.271575,0.275559,0.280795,1.702767,0.008674,1.690333,...,2,3,3,4,1,2,5,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,0.226386,0.020956,0.177879,0.218677,0.222026,0.247189,0.259817,1.591154,0.091644,1.443904,...,5,3,7,3,3,3,3,3,3,3
243,0.169747,0.017591,0.127376,0.156695,0.168922,0.183273,0.201513,1.497708,0.030063,1.410916,...,5,1,9,3,3,3,3,3,3,3
947,0.804265,0.022589,0.763602,0.790417,0.802467,0.815980,0.870211,1.599288,0.033502,1.437445,...,7,6,7,2,2,2,4,2,2,1
542,0.831731,0.017114,0.767162,0.825092,0.833080,0.837141,0.870008,1.723889,0.025204,1.647237,...,3,2,4,3,3,3,3,3,3,3


In [271]:
dataset_df = df.iloc[:,:-17]
self_report_df = df.iloc[:,-17:]

In [272]:
FEATURE = "Dominance"
X = dataset_df 
y = self_report_df[[FEATURE]]

# Pre-processing

In [ ]:
"""#标准化
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)"""

'#标准化\nscaler = StandardScaler()\nscaler.fit(X)\nX = scaler.transform(X)'

In [ ]:
"""from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

X_ = X
pca = PCA(n_components=50)
pca.fit(X_)
X = pca.fit_transform(X_)
X"""

array([[-4.06310717e+00, -1.83662470e+00, -3.87910976e+00, ...,
         8.58151969e-02,  1.28147055e+00, -4.38304038e-01],
       [ 2.77601858e+01,  6.62522115e+00,  3.94761577e+00, ...,
        -9.71571866e-02,  5.56643623e-02,  3.29163799e-01],
       [-6.20595926e+00,  3.74587742e+00, -7.60855264e+00, ...,
        -1.22073024e+00,  3.54890489e-03, -1.19693631e+00],
       ...,
       [-1.96194814e+00, -2.31689925e+00, -2.13261937e+00, ...,
         1.01718956e+00,  6.62087630e-02, -2.14657616e-01],
       [ 7.24387790e-01, -3.14335792e+00,  3.15374373e+00, ...,
        -1.41713548e+00,  1.65491912e+00,  1.80381206e+00],
       [-1.10949092e+00, -5.44253127e+00,  8.07375986e-01, ...,
        -5.50933150e-01,  3.46243794e-01, -5.02533210e-01]])

In [ ]:
"""from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler()
X = mm.fit_transform(X)
X"""

array([[0.10057199, 0.32476159, 0.08397615, ..., 0.59620653, 0.60527013,
        0.60440275],
       [0.18904044, 0.1944494 , 0.18194487, ..., 1.        , 1.        ,
        1.        ],
       [0.13934606, 0.02129402, 0.16672847, ..., 0.26264007, 0.29380743,
        0.28871436],
       ...,
       [0.04184321, 0.16971553, 0.05376077, ..., 0.64962119, 0.65891376,
        0.67457777],
       [0.07698884, 0.10865998, 0.05715281, ..., 0.40405864, 0.40642234,
        0.40259504],
       [0.14059431, 0.0472221 , 0.15693758, ..., 0.42976908, 0.43395377,
        0.43639425]])

# Select feature

In [ ]:
"""from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
a=SelectFromModel(
    GradientBoostingClassifier()
).fit_transform(X,y)
from sklearn.ensemble import  ExtraTreesClassifier
tree = ExtraTreesClassifier()
tree = tree.fit(X,y)

print(tree.feature_importances_)
model = SelectFromModel(tree , prefit=True)
X = model.transform(X)"""

'from sklearn.feature_selection import SelectFromModel\nfrom sklearn.ensemble import GradientBoostingClassifier\na=SelectFromModel(\n    GradientBoostingClassifier()\n).fit_transform(X,y)\nfrom sklearn.ensemble import  ExtraTreesClassifier\ntree = ExtraTreesClassifier()\ntree = tree.fit(X,y)\n\nprint(tree.feature_importances_)\nmodel = SelectFromModel(tree , prefit=True)\nX = model.transform(X)'

In [273]:

from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics

random_state=4

#X = dataset_df 
#y = self_report_df["Arousal"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.05, random_state = random_state)

In [274]:

# 训练模型
model = OneVsRestClassifier(svm.SVC(kernel='rbf',max_iter=20000,probability=True,random_state=random_state))
print("[INFO] Successfully initialize a new model !")
print("[INFO] Training the model…… ")
clt = model.fit(X_train,y_train)
print("[INFO] Model training completed !")
# 保存训练好的模型，下次使用时直接加载就可以了
#joblib.dump(clt,"F:/python/model/conv_19_80%.pkl")
#print("[INFO] Model has been saved !")

proba = clt.predict_proba(X_test)
#print(proba)
y_test_pred = clt.predict(X_test)
ov_acc = metrics.accuracy_score(y_test_pred,y_test)
print("overall accuracy: %f"%(ov_acc))
print("===========================================")
acc_for_each_class = metrics.precision_score(y_test,y_test_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("===========================================")
avg_acc = np.mean(acc_for_each_class)
print("average accuracy:%f"%(avg_acc))

f1 = metrics.f1_score(y_test,y_test_pred,average='micro')
print("f1 score:\n",f1)
print("===========================================")
recall = metrics.recall_score(y_test,y_test_pred,average='micro')
print("recall score:\n",recall)
print("===========================================")
from imblearn.metrics import geometric_mean_score
g_mean = geometric_mean_score(y_test, y_test_pred, average='micro')
print("g-mean score:\n",g_mean)
print("===========================================")

print(y_test_pred)
print(y_test)


[INFO] Successfully initialize a new model !
[INFO] Training the model…… 
[INFO] Model training completed !
overall accuracy: 0.664179
acc_for_each_class:
 [0.49180328 0.6440678  0.66860465 0.66666667 0.63       0.8125
 0.91176471 0.85185185 0.63265306]
average accuracy:0.701101
f1 score:
 0.664179104477612
recall score:
 0.664179104477612
g-mean score:
 0.7976831775655018
[1 6 6 5 5 1 2 3 1 9 5 3 5 5 2 5 2 9 8 3 3 9 3 5 3 8 8 8 5 3 2 3 5 2 3 2 5
 3 2 3 2 2 5 6 3 3 2 3 1 7 7 4 3 3 5 6 2 1 8 3 3 3 1 5 9 3 2 8 5 1 3 3 1 3
 5 7 1 3 5 4 3 4 7 8 3 9 1 3 3 1 3 3 2 3 1 3 3 3 3 3 2 5 3 2 1 5 9 3 9 5 3
 3 5 3 2 2 9 7 3 9 5 2 3 5 5 7 5 5 3 3 4 3 5 5 9 2 5 3 3 3 3 5 1 1 1 2 7 9
 5 5 3 4 2 2 5 3 3 5 3 3 5 4 1 1 2 2 3 2 9 4 3 3 3 5 9 9 9 2 1 2 5 5 3 4 3
 1 1 5 5 3 5 3 6 4 8 3 7 9 3 2 7 3 9 5 1 2 5 3 5 1 3 5 3 3 3 3 6 3 5 1 5 7
 9 3 5 5 3 5 7 1 1 1 3 5 1 3 5 2 3 9 3 2 9 2 3 9 3 7 7 7 5 3 5 5 1 3 4 1 2
 3 7 7 8 3 8 3 3 1 9 6 3 5 1 2 3 9 3 2 7 1 5 3 9 2 9 9 8 3 1 2 2 5 7 3 3 7
 1 4 3 5 3 7 8 3 5 3 8 3

In [275]:
all_proba = clt.predict_proba(X)
all_proba

array([[1.73164223e-02, 8.41055724e-02, 8.32236774e-01, ...,
        2.42911949e-02, 3.91730353e-02, 3.15220789e-07],
       [1.40584416e-03, 6.17218535e-01, 1.94046503e-01, ...,
        1.24511982e-01, 4.65674313e-06, 1.44603666e-02],
       [1.47430580e-02, 5.30088151e-04, 1.27619214e-01, ...,
        1.57862078e-03, 3.49960556e-03, 5.66743326e-05],
       ...,
       [1.97147431e-03, 4.76637574e-03, 4.13602348e-02, ...,
        4.39047847e-01, 1.86051796e-07, 6.08676697e-06],
       [2.76787454e-02, 9.51392695e-03, 5.06229622e-04, ...,
        6.90841020e-04, 1.98631256e-05, 2.13721127e-05],
       [2.58359804e-02, 5.85291490e-02, 5.65088073e-04, ...,
        5.03621826e-04, 4.28425171e-03, 5.41608618e-03]])

In [281]:
high_confidence_samples_list=[]
low_confidence_samples_list=[]

n1=0
n2=0

for i in range(len(X)):
  sample = df.iloc[[i]]
  proba = all_proba[i]
  max_proba = max(proba)
  label = y.iloc[[i]]
  #print(label)
  
  #print(max( proba))
  if(max_proba>=0.63):
    n1=n1+1
    print("HIGHNO."+str(n1),sample)
    high_confidence_samples_list.append(sample)
  else:
    n2=n2+1
    print("LOWNO."+str(n2),sample,max(proba))
    low_confidence_samples_list.append(sample)

流式输出内容被截断，只能显示最后 5000 行内容。

     Excite-Not Excite  Serious-Not Serious  Smooth-Not Smooth  \
935                  1                    4                  1   

     Bright-Not Bright  Eminent-Not Eminent  Close-Not Close  Equal-Not Equal  
935                  4                    5                1                1  

[1 rows x 465 columns]
LOWNO.5174      Position_x_mean  Position_x_std  Position_x_min  Position_x_25%  \
380         0.201429        0.051035        0.083726          0.1485   

     Position_x_50%  Position_x_75%  Position_x_max  Position_y_mean  \
380        0.234844        0.241309        0.248437         1.639495   

     Position_y_std  Position_y_min  ...  Valence  Arousal  Dominance  \
380        0.022577        1.612605  ...        5        3          7   

     Excite-Not Excite  Serious-Not Serious  Smooth-Not Smooth  \
380                  3                    3                  3   

     Bright-Not Bright  Eminent-Not Eminent  Close-Not Close  Equal-Not Eq

In [282]:
len(high_confidence_samples_list)

5386

In [283]:
len(low_confidence_samples_list)

5330

In [284]:
high_confidence_df = pd.concat(high_confidence_samples_list)
low_confidence_df = pd.concat(low_confidence_samples_list)



high_confidence_df.to_csv('/content/drive/MyDrive/Emotion-RST/vrchat_emotion_data2022/SELECTED_DATA/high_confidence_'+FEATURE+'_sample.csv')
low_confidence_df.to_csv('/content/drive/MyDrive/Emotion-RST/vrchat_emotion_data2022/SELECTED_DATA/low_confidence_'+FEATURE+'_sample.csv')
